In [1]:
from python.simulation import Simulation
import pandas as pd
import os

In [2]:
help(Simulation)

Help on class Simulation in module python.simulation:

class Simulation(python.wrapper.python_lbm.PythonClient)
 |  Method resolution order:
 |      Simulation
 |      python.wrapper.python_lbm.PythonClient
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, lbmFile)
 |      Load a simulation from lbm file
 |      
 |      Arguments:
 |          lbmFile {str} -- File system path to the lbm file
 |  
 |  get_average_names(self)
 |      List the names of the time averaged measurement areas
 |      
 |      Returns:
 |          list -- List of name strings
 |  
 |  get_averages(self, avg_type)
 |      Get the time averaged values measured during the simulation
 |      
 |      Arguments:
 |          avg_type {str} -- temperature, velocity or flow
 |      
 |      Returns:
 |          DataFrame -- A dataframe of measured values for all areas
 |  
 |  get_boundary_condition_names(self)
 |      Get a list of named boundary 

In [3]:
sim = Simulation(f'{os.getcwd()}/problems/ocp/project.lbm')

In [4]:
sim.get_boundary_condition_names()

['P02HDZ01',
 'P02HDZ02',
 'P02HDZ03',
 'P02HDZ04',
 'P02R01C01SRV01',
 'P02R01C01SRV02',
 'P02R01C01SRV03',
 'P02R01C02SRV01',
 'P02R01C02SRV02',
 'P02R01C02SRV03',
 'P02R01C03SRV01',
 'P02R01C03SRV02',
 'P02R01C03SRV03',
 'P02R01C04SRV01',
 'P02R01C04SRV02',
 'P02R01C04SRV03',
 'P02R01C05SRV01',
 'P02R01C05SRV02',
 'P02R01C05SRV03',
 'P02R01C06SRV01',
 'P02R01C06SRV02',
 'P02R01C06SRV03',
 'P02R01C07SRV01',
 'P02R01C07SRV02',
 'P02R01C07SRV03',
 'P02R01C08SRV01',
 'P02R01C08SRV02',
 'P02R01C08SRV03',
 'P02R01C09SRV01',
 'P02R01C09SRV02',
 'P02R01C09SRV03',
 'P02R01C10SRV01',
 'P02R01C10SRV02',
 'P02R01C10SRV03',
 'P02R02C01SRV01',
 'P02R02C01SRV02',
 'P02R02C01SRV03',
 'P02R02C02SRV01',
 'P02R02C02SRV02',
 'P02R02C02SRV03',
 'P02R02C03SRV01',
 'P02R02C03SRV02',
 'P02R02C03SRV03',
 'P02R02C04SRV01',
 'P02R02C04SRV02',
 'P02R02C04SRV03',
 'P02R02C05SRV01',
 'P02R02C05SRV02',
 'P02R02C05SRV03',
 'P02R02C06SRV01',
 'P02R02C06SRV02',
 'P02R02C06SRV03',
 'P02R02C07SRV01',
 'P02R02C07SRV02'

In [5]:
sim.get_boundary_conditions('P02R04C07SRV03')

,type,temperature,velocity,normal,rel_pos,tau1,tau2,lambda
714,VoxelType.INLET_RELATIVE,7.436508,"[-0.05, 0.0, 0.0]","[-1, 0, 0]","[25, 0, 0]",533.333313,483.383698,0.0
713,VoxelType.INLET_ZERO_GRADIENT,NaN,"[-0.05, 0.0, 0.0]","[1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0


In [6]:
sim.set_time_averaging_period(5.0)
sim.get_time_step()

0.0018193715950474143

In [7]:
sim.get_boundary_conditions()

,type,temperature,velocity,normal,rel_pos,tau1,tau2,lambda
0,VoxelType.FLUID,NaN,"[nan, nan, nan]","[0, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
1,VoxelType.WALL,NaN,"[nan, nan, nan]","[1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
2,VoxelType.WALL,NaN,"[nan, nan, nan]","[-1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
3,VoxelType.WALL,NaN,"[nan, nan, nan]","[0, 1, 0]","[0, 0, 0]",0.000000,0.000000,0.0
4,VoxelType.WALL,NaN,"[nan, nan, nan]","[1, 1, 0]","[0, 0, 0]",0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...
752,VoxelType.INLET_RELATIVE,7.436508,"[0.05, 0.0, 0.0]","[1, 0, 0]","[-25, 0, 0]",533.333313,483.383698,0.0
753,VoxelType.INLET_ZERO_GRADIENT,NaN,"[0.05, 0.0, 0.0]","[-1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
754,VoxelType.INLET_RELATIVE,7.436508,"[0.05, 0.0, 0.0]","[1, 0, 0]","[-25, 0, 0]",533.333313,483.383698,0.0
755,VoxelType.INLET_ZERO_GRADIENT,NaN,"[0.05, 0.0, 0.0]","[-1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0


In [8]:
# Server mass [kg]
M = 8
# Overall specific heat [J/(kg*K)]
cp_eff = 400
# Specific heat capacity of air [J/(kg*K)]
# https://www.engineeringtoolbox.com/air-specific-heat-capacity-d_705.html
cp_air = 1000
# Overall effective heat transfer coefficient [J/(s*m²*K)]
# https://www.engineeringtoolbox.com/convective-heat-transfer-d_430.html
h = 40
# Server surface area [m²]
A = 0.15
# Average airflow rate through server [kg/s]
q_dot = 0.005 # [m³/s]
rho_air = 1.324 # [kg/m³]
m_dot = q_dot * rho_air
# Time constants [s]
tau1 = M*cp_eff/(h*A)
tau2 = M*cp_eff/(m_dot*cp_air)
lambda1 = 0

In [9]:
# Chassi fan specs from PFR0812DHE fan datasheet
# Max input power in W
Pmax = 25.2
# Max RPM
Nmax = 11000.0
# Operational speed (RPM)
Nop = 8000.0
# Max air flow in CFM
Qmax = 109.7
# Fans per server
Nfans = 2
# Operational power from cube law of fans
Pop = Pmax / (Nmax / Nop)**3 * Nfans
# Assume fan volumetric flow is proportional to power 
# around operating point
Qop = Qmax / (Pmax / Pop)
# Calculate flow in m^3/s per RPM at
Q_per_RPM =  Qop * 0.3048**3 / 60 / Nop

In [10]:
# Kinematic viscosity of air (m^2/s)
nu = 1.568e-5
# Thermal conductivity (kW/m K)
k = 2.624e-5
# Prandtl number of air
Pr = 0.707
# Calculate the expected temperature jump across the servers (convert to kW)
def deltaT(p, q):
    return (p / 1000 * nu) / (q * k * Pr)

In [11]:
srvs = []
# 3 servers per chassi, 10 chassis per rack, 12 racks
for rack in range(1, 13):
    for chassi in range(1, 11):
        for srv in range(1, 4):
            pad = lambda i : str(i).zfill(2)
            srvs += [f'P02R{pad(rack)}C{pad(chassi)}SRV{pad(srv)}']

In [12]:
# Temperatures and volumetric flows should be set from some thermal model
vol_flows = [2000 * Q_per_RPM for _ in range(0, len(srvs))]
p = 45
temps = [deltaT(p, q) for q in vol_flows]
sim.set_boundary_conditions(srvs, temps, vol_flows)

In [13]:
sim.get_boundary_conditions()

,type,temperature,velocity,normal,rel_pos,tau1,tau2,lambda
0,VoxelType.FLUID,NaN,"[nan, nan, nan]","[0, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
1,VoxelType.WALL,NaN,"[nan, nan, nan]","[1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
2,VoxelType.WALL,NaN,"[nan, nan, nan]","[-1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
3,VoxelType.WALL,NaN,"[nan, nan, nan]","[0, 1, 0]","[0, 0, 0]",0.000000,0.000000,0.0
4,VoxelType.WALL,NaN,"[nan, nan, nan]","[1, 1, 0]","[0, 0, 0]",0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...
752,VoxelType.INLET_RELATIVE,3.819558,"[0.025069064, 0.0, 0.0]","[1, 0, 0]","[-25, 0, 0]",533.333313,483.383698,0.0
753,VoxelType.INLET_ZERO_GRADIENT,3.819558,"[0.031336326, -0.0, -0.0]","[-1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0
754,VoxelType.INLET_RELATIVE,3.819558,"[0.031336326, 0.0, 0.0]","[1, 0, 0]","[-25, 0, 0]",533.333313,483.383698,0.0
755,VoxelType.INLET_ZERO_GRADIENT,3.819558,"[0.031336326, -0.0, -0.0]","[-1, 0, 0]","[0, 0, 0]",0.000000,0.000000,0.0


In [14]:
sim.get_time()

datetime.datetime(1970, 1, 1, 1, 0)

In [15]:
sim.run(60.0)

In [16]:
sim.get_time()

datetime.datetime(1970, 1, 1, 1, 1, 0, 1036)

In [17]:
sim.get_averages("temperature")

,time,P02HDZ01_out,P02HDZ01_in,P02HDZ03_out,P02HDZ03_in,P02HDZ04_out,P02HDZ04_in,P02HDZ02_out,P02HDZ02_in,sensors_racks_04_to_06_in_b,...,sensors_racks_07_to_09_in_t,sensors_racks_04_to_06_out_b,sensors_racks_04_to_06_out_m,sensors_racks_04_to_06_out_t,sensors_racks_10_to_12_out_b,sensors_racks_10_to_12_out_m,sensors_racks_10_to_12_out_t,sensors_racks_01_to_03_out_b,sensors_racks_01_to_03_out_m,sensors_racks_01_to_03_out_t
0,1970-01-01 01:00:00.001819,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1970-01-01 01:00:05.003270,18.601049,19.643934,18.614223,19.663074,18.622864,19.651440,18.590088,19.640093,20.218313,...,20.076986,21.077663,22.736769,22.745058,22.226925,21.859541,22.251276,21.281242,21.912857,21.999279
2,1970-01-01 01:00:10.004721,18.562853,20.255148,18.566135,20.863766,18.578369,20.542683,18.549526,20.636812,19.071039,...,19.989933,21.553968,23.298534,23.262857,22.391224,22.452209,22.945253,21.315331,22.374441,22.637156
3,1970-01-01 01:00:15.006172,18.552834,21.349211,18.559052,21.922964,18.579140,21.424181,18.544464,21.454475,18.863022,...,19.961519,22.540640,23.455769,23.213242,22.981369,22.411301,22.797497,21.391851,22.390297,22.600529
4,1970-01-01 01:00:20.007622,18.560080,21.791763,18.559784,22.156317,18.574047,21.972532,18.541498,21.810766,18.841896,...,19.987652,23.085733,23.464289,23.337217,23.002043,22.492094,22.855492,21.541624,22.463995,22.883446
5,1970-01-01 01:00:25.009073,18.567913,22.158157,18.563051,22.241570,18.573851,22.188105,18.540119,22.026871,18.825016,...,20.950705,23.472631,23.262663,23.629814,23.365154,22.559036,22.934519,21.596643,22.463514,22.797279
6,1970-01-01 01:00:30.010524,18.560894,22.325161,18.561731,22.406898,18.583153,22.333353,18.538805,22.132322,18.806479,...,20.878319,23.617870,23.142433,23.533609,23.544722,22.563322,22.988949,21.727161,22.544184,23.008171
7,1970-01-01 01:00:35.011975,18.564737,22.387636,18.557171,22.499416,18.578545,22.412098,18.542749,22.301596,18.815454,...,21.332430,23.533875,23.276741,23.578066,23.720894,22.575468,23.111040,21.874006,22.521101,22.972269
8,1970-01-01 01:00:40.013426,18.559526,22.461853,18.557648,22.554310,18.583712,22.546888,18.548012,22.457180,18.835342,...,21.412495,23.386940,23.394590,23.600519,23.773996,22.523741,23.289127,22.022093,22.542274,23.041388
9,1970-01-01 01:00:45.014877,18.555788,22.458591,18.558863,22.593435,18.578699,22.582764,18.544739,22.494762,18.845985,...,21.514851,23.291847,23.461374,23.508728,23.817951,22.581432,23.322021,22.003332,22.627003,23.090351


In [18]:
# Temperatures and volumetric flows should be set from some thermal model
vol_flows = [2000 * Q_per_RPM for _ in range(0, len(srvs))]
temps = [deltaT(1000, q) for q in vol_flows]
sim.set_boundary_conditions(srvs, temps, vol_flows)

In [19]:
sim.run(60.0)

In [20]:
sim.get_averages("temperature")[['time', 'sensors_racks_10_to_12_out_m']]

,time,sensors_racks_10_to_12_out_m
0,1970-01-01 01:00:00.001819,0.000000
1,1970-01-01 01:00:05.003270,21.859541
2,1970-01-01 01:00:10.004721,22.452209
3,1970-01-01 01:00:15.006172,22.411301
4,1970-01-01 01:00:20.007622,22.492094
5,1970-01-01 01:00:25.009073,22.559036
6,1970-01-01 01:00:30.010524,22.563322
7,1970-01-01 01:00:35.011975,22.575468
8,1970-01-01 01:00:40.013426,22.523741
9,1970-01-01 01:00:45.014877,22.581432
